In [1]:
from bs4 import BeautifulSoup
import webbrowser
import requests
import pandas as pd
import cv2 
import urllib.request as urli
import os 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
import pickle
import time 



options = Options()
options.add_argument("--headless")

# Load listings

In [2]:
url_listing = "http://data.insideairbnb.com/germany/bv/munich/2021-12-24/data/listings.csv.gz"
listings = pd.read_csv(url_listing)
prefix = "https://airbnb.de"
suffix = "/photos"
urls = listings["listing_url"]
ids = listings["id"]


In [3]:
webbrowser.open(urls[0])

True

In [4]:
id_dict = dict()
for id, url in tqdm(zip(ids, urls), total = len(ids)):    
    id_list = []
    try:
        #driver = webdriver.Chrome("/Users/dmnk/Documents/Dubair_Trash/chromedriver 2", options = options)
        driver = webdriver.Chrome("D:/Sonstiges/chromedriver.exe", options = options)
        driver.get(url+suffix)

        body = driver.find_element_by_tag_name("body")
        for i in range(40):
            body.send_keys(Keys.TAB)
        
        time.sleep(1)

        for i in range(40):
            body.send_keys(Keys.TAB)            

        html = driver.page_source
        html_content = BeautifulSoup(html, "html.parser")
        gallery_items = html_content.findAll("img", {"class": "_6tbg2q"})
    except:
        continue
    for i, item in enumerate(gallery_items[5:]):
        try:
            img_link = item.get("src")
            img_title = item.get("alt")
            path_name = "D:/Uni/WiSe21-22/Statistical and Deep Learning/image_scraping_munich/"+str(id)+"_"+str(i)+".png"
            if os.path.exists(path_name):
                id_list.append(img_title)
                continue
            else:
                urli.urlretrieve(img_link, path_name)
                id_list.append(img_title)
            
        except:
            print("ups - fail")
            continue

    id_dict[id] = id_list
meta_images = open("D:/Uni/WiSe21-22/Statistical and Deep Learning/image_scraping_munich/data.pkl", "wb")
pickle.dump(id_dict, meta_images)
meta_images.close

  0%|          | 0/4995 [00:00<?, ?it/s]C:\Users\49173\anaconda3\envs\DL_tabnet\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  
C:\Users\49173\anaconda3\envs\DL_tabnet\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if __name__ == '__main__':
 56%|█████▋    | 2822/4995 [10:09:51<6:23:42, 10.59s/it] 

In [5]:
import json

with open('labels_raw.json', 'w') as fp:
    json.dump(id_dict, fp)

Resize images to same shape

In [9]:
resized_path = "D:/Uni/WiSe21-22/Statistical and Deep Learning/images_resized/"

dir = os.listdir("D:/Uni/WiSe21-22/Statistical and Deep Learning/image_scraping")

img_names = pd.Series(dir).str.split(".").str[0].values

for img_dir, resized_img_dir in tqdm(zip(dir, img_names), total = len(dir)):
  img_dir_tmp = "D:/Uni/WiSe21-22/Statistical and Deep Learning/image_scraping/"+ img_dir
  img_tmp = cv2.imread(img_dir_tmp)
  img_tmp = cv2.resize(img_tmp, dsize=(256, 256))
  path_tmp = resized_path + resized_img_dir + ".png"
  cv2.imwrite(path_tmp, img_tmp)

100%|█████████▉| 86889/86890 [34:07<00:00, 42.43it/s]  


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
